In [25]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import nltk
from fuzzywuzzy import fuzz

def max_f(l):   #to get the highest percentage fuzzy match in a list 
    m=l[0]
    for i in l:
        if(i>m):
            m=i
    return m
def pos(l,num):     #to get the position of a number in a list
    for i in range(len(l)):
        if(l[i]==num):
            return i

nltk.download('stopwords')
from nltk.corpus import stopwords    #importing stopwords
stop=stopwords.words('english')      #creating a list of stopwords
nltk.download('punkt')

#loading csv files of Narration and ledger Narration to pandas dataframe
ln=pd.read_csv('/Users/daksha.singhal/Desktop/Copy of Daksha IDA Master Bank Ledger.csv',encoding = "ISO-8859-1", low_memory=False)
n=pd.read_csv('/Users/daksha.singhal/Desktop/Copy of 2305 Master Bank Statement.csv',encoding = "ISO-8859-1", low_memory=False)

#OPTIONAL
#selecting only the rows of IDAA Bank 
ln=ln.loc[ln['Bank']=='IDAA']
n=n.loc[n['Bank']=='IDAA']

#replacing NA with empty spaces
ln["Ledger Narration 1"].fillna("", inplace = True) 
n["Narration 1"].fillna("", inplace = True) 

#changing datatype to str
n['Narration 1']=n['Narration 1'].astype('str')
ln['Ledger Narration 1']=ln['Ledger Narration 1'].astype('str')
#changing datatype to float
ln['Ledger EY Amount']=ln['Ledger EY Amount'].astype('float')
n['EY Amount']=n['EY Amount'].astype('float')


#Data Preprocessing
stop=[i for i in stop if len(i)>1]
ln['Ledger Narration 1'] =ln['Ledger Narration 1'].apply(lambda x: " ".join(x.lower() for x in x.split())) 
ln['Ledger Narration 1']=ln['Ledger Narration 1'].str.replace('[^\w\s]',' ')
ln['Ledger Narration 1'] =ln['Ledger Narration 1'].apply(lambda x: " ".join(x for x in x.split() if x not in (stop)))
ln['Ledger Narration 1'] = ln['Ledger Narration 1'].apply(lambda x: " ".join(x for x in x.split() if x.isalpha()==True))

unq_col=list(ln['Ledger Narration 1'].unique())
unq_words=[]
for i in unq_col:
    for j in i.split():
        unq_words.append(j)

unq_words=list(set(unq_words))

nn=n['Narration 1']
n['Narration 1'] =n['Narration 1'].apply(lambda x: " ".join(x.lower() for x in x.split())) 
n['Narration 1']=n['Narration 1'].str.replace('[^\w\s]',' ')
freq = list(set(pd.Series(' '.join(ln['Narration 1']).split()).value_counts()[:5].index)-unq_words) ##common words removal
n['Narration 1'] = n['Narration 1'].apply(lambda x: " ".join(x for x in x.split() if x not in (stop+freq-)))
n['Narration 1'] = n['Narration 1'].apply(lambda x: " ".join(x for x in x.split() if x.isalpha()==True))


#creating a list of ledger narrations whose amounts match with narration 
le_narration=[]
for i in n.index:
    le_narration.append(ln.loc[ln['Ledger EY Amount']==-n['EY Amount'][i]])


stopwords_narration=n['Narration 1']
regex = RegexpTokenizer("[\w']+")
percent=[]
for i in range((len(le_narration))):
    percent.append([])
    for j in le_narration[i].index:
        str2=le_narration[i]['Ledger Narration 1'][j]
        str1=stopwords_narration[i]
        percent[i].append(fuzz.token_sort_ratio(str1,str2))

final_percent=[]
ind=[]
for i in range(len(percent)):
    if(len(percent[i])!=0):
        final_percent.append(max_f(percent[i]))
    else:
        ind.append(i)
        final_percent.append('Amounts didnt match')

final_ln=[]
for i in range(len(percent)):
    if(len(percent[i])!=0):
        final_ln.append(le_narration[i].loc[list(le_narration[i].index.values)[percent[i].index(final_percent[i])],])
    else:
        final_ln.append('')

new_ln = pd.DataFrame(columns=list(ln.columns))
for i in range(len(final_ln)):
    if(i not in ind):
        new_ln=new_ln.append(final_ln[i])

new_ln=new_ln.reset_index(drop=True)
narration1=n.drop(ind,axis=0)
narration1.index=range(len(narration1))

final_percent=[i for i in final_percent if i!='Amounts didnt match']
narration1['score']=final_percent

#creating a dataframe which merges narration and the corresponding Ledger narration with the fuzzy match score
result=pd.merge(narration1,new_ln, left_index=True, right_index=True)


#Only selecting the rows having a fuzzy match of 60 or more
result=result.loc[result['score']>=60]

#Exporting the output in csv file
result.to_csv('/Users/daksha.singhal/Desktop/result7.csv', encoding='utf-8', index=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\daksha.singhal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daksha.singhal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


PermissionError: [Errno 13] Permission denied: '/Users/daksha.singhal/Desktop/result7.csv'